In [18]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

## Project #1. Flows on the London Subway ##

For this project we will use data on the London subway. The data is organized as follows:
- file ''connections'' contains data on the direct links between two stations; for instance, we can go from Station 11 to Station 163 (and vice versa) by taking line 1; it will take us 1 minute;
- file ''stations'' contains information on identifiers of each station;
- file ''stations_restricted'' contains a subset of the stations from stations.csv

From this information, we can construct a network as follows:
- the nodes are the stations;
- there is an arc between two nodes if there is a direct link between the two corresponding stations in the file ''connections'' (assume every connection corresponds to two arcs in opposite directions); for instance, there is an arc (11,163), and arc (163,11), but no arc (1,170);
- define capacities as follows:
1.   Generate a random number between 5 and 10 for each line: this is the capacity of the line.
2.   The capacity of an arc between two stations has the same capacity as the sum of the capacities of the lines that link them. For instance, the stations of King's Cross St. Pancras (#145) and Euston Square (#90) are linked by lines 3,6,8. Let's say we randomly sampled the capacity of 6,8,4 for lines 3,6,8. The capacity of the arc (145,90) is 6+8+4=18. Similarly, the capacity of the arc (90,145) is also 6+8+4=18.

The project is composed of 4 questions, all to be solved using Linear Program and to be completed in this order:

1. We want to compute how many passengers can transit on average from the north-east part of the city center, to the south-west. To do so, compute a maximum flow between King's Cross St. Pancras (#145) and Hammersmith (#110) on the instance in the file ''connections'' (ignore the time it takes to go between stations).
2. We want to understand how many people can be evacuated from the city center within 18 minutes. To do so, restrict to the stations in the file ''stations_restricted'' (and the corresponding arcs). Compute the maximum flow over time between King's Cross St. Pancras (#145) and Victoria (#273) for T=18.
3. Consider again the restriction to the stations in the file ''stations_restricted''. For security reasons, in the stations Oxford Circus (#192) and Embankment (#87), at any given time there can be at most 20 passengers in transit. Solve problem 2) above under this additional restriction.
4. **4) Will not be evaluated unless parts 1),2),3) have been correctly solved** Choose an improvement to the line and write a linear program that finds the *best* way to realize your improvement. For instance, you could assume you have a budget of 10 units of additional capacity to be added to the network, and you want to decide where to add them so that the maximum flow in part 1) above increases the most (it is ok to have an improvement that requires an integer solution, but the solution you get is fractional).





In [19]:
import pandas as pd
import numpy as np
from scipy.optimize import linprog
from collections import defaultdict
import gurobipy as gp
from gurobipy import Model, GRB

# Load the datasets
connections_df = pd.read_csv('connections.csv')
stations_df = pd.read_csv('stations.csv')


In [20]:
# Assign random capacities between 5 and 10 for each unique line
np.random.seed(0)  # for reproducibility
unique_lines = connections_df['line'].unique()
line_capacities = {line: np.random.randint(5, 11) for line in unique_lines}
connections_df['line_capacity'] = connections_df['line'].map(line_capacities)

connections_df.head()

,link name,station1,station2,line,time,distance(miles),line_capacity
0,338,1,73,10,2.0,0.612553,9
1,339,1,234,10,4.0,1.170653,9
2,340,1,265,10,3.0,1.214326,9
3,73,2,156,3,2.0,0.379161,10
4,74,2,263,3,4.0,0.314278,10


In [29]:
# Define source and sink
source = 145  # King's Cross St. Pancras
sink = 110    # Hammersmith

# Extract relevant information for the network flow problem
edges = connections_df[['station1','station2','line_capacity']].copy()
edges.columns = ['start','end','capacity'] # rename for clarity

# Set up model
m = Model("MaxFlow")

# Create variables for flow on each edge
flow = {}
for i,row in edges.iterrows():
    flow[(row['start'],row['end'])] = m.addVar(ub=row['capacity'], name=f"flow_{row['start']}_{row['end']}")
    flow[(row['end'],row['start'])] = m.addVar(ub=row['capacity'], name=f"flow_{row['end']}_{row['start']}") 

# Set the objective: max the flow out of the source node (station 145)
out_flow = gp.quicksum(flow[(source,v)] for u,v in flow if u == source)
in_flow = gp.quicksum(flow[(u,source)]for u,v in flow if v == source)
m.setObjective(out_flow - in_flow, GRB.MAXIMIZE)

# Add flow conservation constraints for all nodes except source and sink
nodes = set(edges['start']).union(set(edges['end']))
for node in nodes - {source, sink}:
    m.addConstr(
        gp.quicksum(flow[(u, node)] for u,v in flow if v == node) ==
        gp.quicksum(flow[(node, v)] for u,v in flow if u == node),
        name=f"flow_conservation_{node}"
    )

# Optimize the model
m.optimize()

# Result
if m.status == GRB.OPTIMAL:
    max_flow = m.ObjVal
    flow_values = {edge: var.x for edge, var in flow.items()}
    max_flow, flow_values
else: max_flow, flow_values = None, None

max_flow, flow_values

Gurobi Optimizer version 11.0.3 build v11.0.3rc0 (mac64[x86] - Darwin 23.5.0 23F79)

CPU model: Intel(R) Core(TM) i9-9880H CPU @ 2.30GHz
Thread count: 8 physical cores, 16 logical processors, using up to 16 threads

Optimize a model with 320 rows, 866 columns and 1482 nonzeros
Model fingerprint: 0x84799f77
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [5e+00, 1e+01]
  RHS range        [0e+00, 0e+00]
Presolve removed 274 rows and 670 columns
Presolve time: 0.01s
Presolved: 46 rows, 196 columns, 374 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    2.9999975e+01   4.300000e+01   0.000000e+00      0s
Extra simplex iterations after uncrush: 1
      48    2.5000000e+01   0.000000e+00   0.000000e+00      0s

Solved in 48 iterations and 0.02 seconds (0.00 work units)
Optimal objective  2.500000000e+01


(25.0,
 {(1, 73): 0.0,
  (73, 1): 8.0,
  (1, 234): 0.0,
  (234, 1): 0.0,
  (1, 265): 8.0,
  (265, 1): 0.0,
  (2, 156): 0.0,
  (156, 2): 0.0,
  (2, 263): 0.0,
  (263, 2): 0.0,
  (3, 263): 0.0,
  (263, 3): 0.0,
  (3, 295): 0.0,
  (295, 3): 0.0,
  (3, 156): 0.0,
  (156, 3): 0.0,
  (4, 70): 0.0,
  (70, 4): 0.0,
  (4, 201): 0.0,
  (201, 4): 0.0,
  (5, 194): 1.0,
  (194, 5): 0.0,
  (5, 252): 0.0,
  (252, 5): 1.0,
  (6, 46): 0.0,
  (46, 6): 0.0,
  (7, 145): 0.0,
  (145, 7): 3.0,
  (7, 188): 3.0,
  (188, 7): 0.0,
  (8, 124): 0.0,
  (124, 8): 0.0,
  (8, 264): 0.0,
  (264, 8): 0.0,
  (9, 31): 0.0,
  (31, 9): 0.0,
  (9, 232): 0.0,
  (232, 9): 0.0,
  (10, 95): 0.0,
  (95, 10): 0.0,
  (10, 128): 0.0,
  (128, 10): 0.0,
  (11, 163): 3.0,
  (163, 11): 0.0,
  (11, 212): 0.0,
  (212, 11): 0.0,
  (11, 83): 0.0,
  (83, 11): 0.0,
  (11, 104): 0.0,
  (104, 11): 5.0,
  (11, 28): 1.0,
  (28, 11): 0.0,
  (11, 249): 0.0,
  (249, 11): 0.0,
  (11, 94): 1.0,
  (94, 11): 0.0,
  (12, 56): 0.0,
  (56, 12): 0.0,
  (12

In [ ]:
# Define source and sink
source = 145  # King's Cross St. Pancras
sink = 110    # Hammersmith

# Extract relevant information for the network flow problem
edges = connections_df[['station1','station2','line_capacity']].copy()
edges.columns = ['start','end','capacity'] # rename for clarity

# Set up model
m = Model("MaxFlow")

# Create variables for flow on each edge
flow = {}
for i,row in edges.iterrows():
    flow[(row['start'],row['end'])] = m.addVar(ub=row['capacity'], name=f"flow_{row['start']}_{row['end']}")
    flow[(row['end'],row['start'])] = m.addVar(ub=row['capacity'], name=f"flow_{row['end']}_{row['start']}")

# Set the objective: max the flow out of the source node (station 145)
out_flow = gp.quicksum(flow[(source,v)] for u,v in flow if u == source)
in_flow = gp.quicksum(flow[(u,source)]for u,v in flow if v == source)
m.setObjective(out_flow - in_flow, GRB.MAXIMIZE)

# Add flow conservation constraints for all nodes except source and sink
nodes = set(edges['start']).union(set(edges['end']))
for node in nodes - {source, sink}:
    m.addConstr(
        gp.quicksum(flow[(u, node)] for u,v in flow if v == node) ==
        gp.quicksum(flow[(node, v)] for u,v in flow if u == node),
        name=f"flow_conservation_{node}"
    )

# Optimize the model
m.optimize()

# Result
if m.status == GRB.OPTIMAL:
    max_flow = m.ObjVal
    flow_values = {edge: var.x for edge, var in flow.items()}
    max_flow, flow_values
else: max_flow, flow_values = None, None

max_flow, flow_values

In [12]:
# Step 2: Set up model
model = Model("MaxFlow")

# Define source and sink
source_station = 145  # King's Cross St. Pancras
sink_station = 110    # Hammersmith


Restricted license - for non-production use only - expires 2025-11-24


In [13]:
# Step 3: Create variables for each arc with capacity limits
flow_vars = {}
for idx, row in arc_df.iterrows():
    station1, station2, capacity = int(row['station1']), int(row['station2']), row['arc_capacity']
    flow_vars[(station1, station2)] = model.addVar(lb=0, ub=capacity, vtype=GRB.CONTINUOUS, name=f"flow_{station1}_{station2}")


In [14]:
# Step 4: Flow conservation constraints
for station in stations_df['Station code']:
    inflow = sum(flow_vars[(i, station)] for i, j in flow_vars if j == station)
    outflow = sum(flow_vars[(station, j)] for i, j in flow_vars if i == station)
    if station == source_station:
        model.addConstr(outflow - inflow == 1, f"source_flow_{station}")
    elif station == sink_station:
        model.addConstr(inflow - outflow == 1, f"sink_flow_{station}")
    else:
        model.addConstr(inflow == outflow, f"flow_conservation_{station}")


In [15]:
# Step 5: Objective function to maximize flow from source
model.setObjective(sum(flow_vars[(source_station, j)] for i, j in flow_vars if i == source_station), GRB.MAXIMIZE)

# Step 6: Solve the model
model.optimize()

# Step 7: Extract the maximum flow value
if model.status == GRB.OPTIMAL:
    max_flow_value = model.objVal
    print("Maximum Flow from King's Cross St. Pancras to Hammersmith:", max_flow_value)
else:
    print("No optimal solution found.")

Gurobi Optimizer version 11.0.3 build v11.0.3rc0 (mac64[x86] - Darwin 23.5.0 23F79)

CPU model: Intel(R) Core(TM) i9-9880H CPU @ 2.30GHz
Thread count: 8 physical cores, 16 logical processors, using up to 16 threads

Optimize a model with 332 rows, 377 columns and 754 nonzeros
Model fingerprint: 0xe9682315
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [5e+00, 3e+01]
  RHS range        [1e+00, 1e+00]
Presolve removed 193 rows and 271 columns
Presolve time: 0.01s

Solved in 0 iterations and 0.01 seconds (0.00 work units)
Infeasible model
No optimal solution found.


Q4.

In [ ]:
import random
from gurobipy import Model, GRB, quicksum

connections_df = pd.read_csv('connections.csv')

# Generate random capacities for each line
line_capacities = {line: random.randint(5, 10) for line in connections_df['line'].unique()}

# Create a dictionary to map arcs and their capacities
arc_capacities = {}
for _, row in connections_df.iterrows():
    line = row['line']
    capacity = line_capacities[line]
    
    # Calculate capacity for the arc
    arc_capacities[(row['station1'], row['station2'])] = arc_capacities.get((row['station1'], row['station2']), 0) + capacity
    arc_capacities[(row['station2'], row['station1'])] = arc_capacities.get((row['station2'], row['station1']), 0) + capacity

# Gurobi model
model = Model("Max_Flow_Improvement")

# Decision variables: additional capacity to add to each arc
additional_capacity = model.addVars(arc_capacities.keys(), lb=0, name="additional_capacity")

# Objective function: maximize the flow
# Calculate the new maximum flow based on the current flow and additional capacities

# Max flow with additional capacity added (placeholder)
max_flow_expression = quicksum(arc_capacities[arc] + additional_capacity[arc] for arc in arc_capacities)
model.setObjective(max_flow_expression, GRB.MAXIMIZE)

# Constraint: total additional capacity must not exceed 10
model.addConstr(quicksum(additional_capacity[arc] for arc in additional_capacity) <= 10, "CapacityConstraint")

# Solve
model.optimize()

# Results
if model.status == GRB.OPTIMAL:
    print("Optimal Additional Capacities:")
    for arc in additional_capacity:
        print(f"Arc {arc}: {additional_capacity[arc].X}")
else:
    print("No optimal solution found.")
